# Exploratory Data Analysis
The goal of this notebook is to explore the nba match details dataset. It has 47 columns, so there's quite a bit to unpack here. This notebook seeks to identify which features are important, and which ones are correlated. Trying to reduce our complexity here

First, lets do some imports

In [ ]:
import pandas as pd
import numpy as np